# DNN evaluation across different sizes for a given p-value threshold

In [ ]:
%matplotlib notebook
import tensorflow as tf
import numpy as np
from pandas_plink import read_plink
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
from sklearn import metrics
from math import sqrt

import random
from sklearn.metrics import roc_curve,roc_auc_score

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
''' Minibatch function'''
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [ ]:
''' Parameters for experiment '''
threshold="0.01"
path_logs="/work/breastcancer/clean_test/logs/"
path_to_files="/work/breastcancer/clean_test/"

In [ ]:
''' getting bim,fam,bed for training,validation and test sets '''
(bim, fam, bed)=read_plink(path_to_files+"train/sig"+threshold)
(bim2, fam2, bed2)=read_plink(path_to_files+"validation/val"+threshold)
(bim3, fam3, bed3)=read_plink(path_to_files+"test/test"+threshold)

path_logs="/work/breastcancer/clean_test/logs/"

print(bim)

print(fam)

''' Creating arrays with optimal data structure and filling missing values with 2--> Homozygous major '''
print("Convertion")
bed=bed.astype('uint8')
print("Compute")
X=bed.compute()
print("Filling Null Data")
X[np.isnan(X)]=2
#validation
print("Convertion")
bed2=bed2.astype('uint8')
print("Compute")
X_val=bed2.compute()
print("Filling Null Data")
X_val[np.isnan(X_val)]=2
#test
print("Convertion")
bed3=bed3.astype('uint8')
print("Compute")
X_test=bed3.compute()
print("Filling Null Data")
X_test[np.isnan(X_test)]=2

''' Preparing data.shape=(individuals,SNP) '''
#train
Y=fam["trait"].astype("int")
Y=Y-1
Xdf=pd.DataFrame(X.T)
Xdf["Y"]=Y

#validation
Y_val=fam2["trait"].astype("int")
Y_val=Y_val-1
Xdf_val=pd.DataFrame(X_val.T)
Xdf_val["Y"]=Y_val

#test
Y_test=fam3["trait"].astype("int")
Y_test=Y_test-1
Xdf_test=pd.DataFrame(X_test.T)
Xdf_test["Y"]=Y_test

''' Getting np arrays '''
x_train=Xdf.drop(['Y'],axis=1).values
y_train=Xdf[['Y']].values

x_val=Xdf_val.drop(['Y'],axis=1).values
y_val=Xdf_val[['Y']].values

x_test=Xdf_test.drop(['Y'],axis=1).values
y_test=Xdf_test[['Y']].values

In [ ]:
''' MODEL '''

''' Inputs tensors '''
tf.reset_default_graph()
X=tf.placeholder(tf.float32,shape=(None,x_train.shape[1]),name="X")
Y=tf.placeholder(tf.float32,shape=(None,1),name="Y")

''' DNN with dropout rate=0.5 and Batch Norm'''

with tf.name_scope("dnn"):
    training = tf.placeholder_with_default(False, shape=(), name='training')
    initializer = tf.contrib.layers.xavier_initializer()
    hidden00_drop= tf.layers.dropout(X, 0.5, training=training)
    hidden01=tf.layers.dense(hidden00_drop, 1000, name="hidden01",activation=None, kernel_initializer=initializer)
    hidden01_norm=tf.layers.batch_normalization(hidden01, training=training, momentum=0.9)
    act_hidden01=tf.nn.leaky_relu(hidden01_norm)
    hidden01_drop = tf.layers.dropout(act_hidden01, 0.5, training=training)
    hidden0=tf.layers.dense(hidden01_drop, 250, name="hidden0",activation=None, kernel_initializer=initializer)
    hidden0_norm=tf.layers.batch_normalization(hidden0, training=training, momentum=0.9)
    act_hidden0=tf.nn.leaky_relu(hidden0_norm)
    hidden0_drop = tf.layers.dropout(act_hidden0, 0.5, training=training)
    hidden1=tf.layers.dense(hidden0_drop, 50, name="hidden1",activation=None, kernel_initializer=initializer)
    hidden1_norm=tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
    act_hidden1=tf.nn.leaky_relu(hidden1_norm)
    hidden1_drop = tf.layers.dropout(act_hidden1, 0.5, training=training)
    hidden1_norm=tf.layers.batch_normalization(hidden1_drop, training=training, momentum=0.9)
    output=tf.layers.dense(  hidden1_norm, 1, name="output_final",activation=None)

''' Log-Loss '''
with tf.name_scope("loss"):
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=output)
    weights = tf.trainable_variables()
    loss=tf.reduce_mean(cross_entropy)
    error=loss

''' Adam Optimizer '''
with tf.name_scope("train"):
    optimizer =tf.train.AdamOptimizer(learning_rate=0.0001,beta1=0.9,beta2=0.999,epsilon=1e-08,use_locking=False,name='Adam')
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        training_op = optimizer.minimize(error)

''' Metrics '''
with tf.name_scope("eval"):
    predicted = tf.nn.sigmoid(output)
    correct_pred = tf.equal(tf.round(predicted), Y)
    acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    _,auc = tf.metrics.auc(labels=Y,predictions=predicted)

saver=tf.train.Saver(max_to_keep=100000)

In [ ]:
''' Train models with different size and record the best per size'''
open('/work/breastcancer/Test_preprocessed_filter7.txt', 'w')
sizes=[10000,20000,30000,x_train.shape[0]]
best_val_size=[]
for size in sizes:
    best_auc=0
    init=tf.global_variables_initializer()
    loc=tf.local_variables_initializer()
    sess = tf.InteractiveSession(config=tf.ConfigProto(device_count={ "CPU": 44}))
    init.run()
    loc.run()
    auc_tab=[]
    accuracy_tab=[]
    loss_tab=[]
    epoch_tab=[]
    auc_tab_val=[]
    accuracy_tab_val=[]
    loss_tab_val=[]
    #Training
    for epoch in range(200):
        iteration=0
        batch_size=512
        for x_batch,y_batch in shuffle_batch(x_train[0:size,:], y_train[0:size,:], batch_size):
            sess.run(training_op,feed_dict={X:x_batch,Y:y_batch,training:True})
            print("%d ITERATION:%d/%d "%(epoch,iteration,len(x_train[0:size,:])//batch_size),end='\r')
            iteration+=1

        loc.run()
        loss_train,acc_train,auc_train=sess.run([loss,acc,auc],feed_dict={X:x_batch,Y:y_batch,training:False})
        print(epoch,"Train accuracy:",acc_train,"Loss:",loss_train,"AUC:",auc_train)
        auc_tab.append(auc_train)
        accuracy_tab.append(acc_train)
        epoch_tab.append(epoch)
        loss_tab.append(loss_train)
        #validation
        loc.run()
        loss_val,acc_val,auc_val=sess.run([loss,acc,auc],feed_dict={X:x_val,Y:y_val,training:False})
        auc_tab_val.append(auc_val)
        accuracy_tab_val.append(acc_val)
        loss_tab_val.append(loss_val)
        if auc_val>best_auc:
            save_path = saver.save(sess,path_logs+"preprocessed"+threshold+"_data_"+size+".ckpt")
            best_auc=auc_val
        print(epoch,"Validation accuracy:",acc_val,"Loss:",loss_val,"AUC:",auc_val)
        print("\n")
    sess.close()
    sess = tf.InteractiveSession(config=tf.ConfigProto(device_count={ "CPU": 44}))
    init.run()
    loc.run()
    #Test score for the best model for AUC score on validation set. This final test is made on the test set 
    saver.restore(sess, path_logs+"preprocessed"+threshold+"_data_"+size+".ckpt")
    auc_test=sess.run(auc,feed_dict={X:x_test,Y:y_test,training:False})
    print(best_auc,auc_test)
    best_val_size.append([best_auc,auc_test])
    sess.close()

In [ ]:
'''Save Results'''
np.savetxt(path_logs+"best_value_test_auc.csv", np.array(best_val_size), delimiter=",",)